In [7]:
import pandas
import os
import json

class OptaData:
    def __init__(self, base_path):
        """
        Initialize training/testing data
        """
        self.base_path = base_path
        self.data=self.load_data()
        
#         self.X, self.P, self.Y, self.goalkeeper, self.defender, self.midfielder, self.forward, self.csvfile  = self.load_data()

    def get_csv_files(self):
        for root, folders, files in os.walk(self.base_path):
            for file in files:
                if '.csv' in file:
                    yield(root + '/' + file)


    def load_data(self):
        """
        Load data from file
        Args:
        Returns:
        """

        X = []; Y = []; P = []; positions={}; goalkeeper = []; defender = []; midfielder = []; forward = []
        
        label_dict = {}
        with open('columns', 'r') as columns_file:
            columns = columns_file.read().splitlines()
        with open('form_columns', 'r') as form_columns_file:
            form_columns = form_columns_file.read().splitlines()

        for csv_file in self.get_csv_files():
            
             
            
            df = pandas.read_csv(csv_file, usecols=columns)
            df = df.fillna(value=0.0)
            replace_outcome = {1.0:1.0, 0.0:-1.0}
            df["outcome"].replace(replace_outcome, inplace=True)

            for game_id in df.game_id.unique():
                df_game = df.loc[df['game_id'] == game_id]

                df_position_team = df_game.loc[df['event_type'] == 'Team set up']
                for index, row in df_position_team.iterrows():
                    positions_id = row['player_position'].replace(' ','').split(',')
                    involved = row['involved'].replace(' ','').split(',')
                    for index, position in enumerate(positions_id):
                        if position != 5:
                            positions[float(involved[index])] = position
                df_substitute = df_game.loc[df['event_type'] == 'Player on']
                for index, row in df_substitute.iterrows():
                    if row['player_position']=='Goalkeeper':
                        positions[float(row['player_id'])]=1
                    elif row['player_position'] == 'Defender':
                        positions[float(row['player_id'])] = 2
                    elif row['player_position'] == 'Midfielder':
                        positions[float(row['player_id'])] = 3
                    elif row['player_position'] == 'Forward':
                        positions[float(row['player_id'])] = 4


                for player_id in df_game.player_id.unique():
                    if player_id == 0.0:
                        continue
                    df_player = df_game.loc[df_game['player_id'] == player_id]
                    features = []
                    for feature in form_columns:
                        total = (df_player[feature]).sum()
                        if total:
                            success = (df_player[feature]*df_player['outcome']).sum()
                            features.append(success/total)
                        else:
                            features.append(0.0)
                    if True:
                        if True:
                            arr = [str(game_id), str(player_id),df_player['game_date'].iloc[0]] + features
                            if positions[player_id] == 1:
                                    goalkeeper.append(arr)
                            elif positions[player_id] == 2:
                                    defender.append(arr)
                            elif positions[player_id] == 3:
                                    midfielder.append(arr)
                            elif positions[player_id] == 4:
                                forward.append(arr)

                            X.append(features)
                            P.append(positions[player_id])
                            #Y.append(player_rating[str(game_id)][str(player_id)])
                        else:
                            print ("Player missing : ", game_id, player_id)
                    else:
                        print ("Game missing : ", game_id)
            yield X, P, Y, goalkeeper, defender, midfielder, forward, csv_file



od = OptaData('./data1/Full Datasets - Opta/MLS')


In [8]:
import csv as csv
for X,P,Y,goalkeeper,defender,midfielder,forward,csv_file in od.data:
    print csv_file
    csv_file=csv_file.replace('/','')
    with open("gk"+csv_file, 'w') as csvfile1:
        csvwriter = csv.writer(csvfile1)
        csvwriter.writerows(goalkeeper)
    with open("def"+csv_file, 'w') as csvfile2:
        csvwriter = csv.writer(csvfile2)
        csvwriter.writerows(defender)
    with open("fwd"+csv_file, 'w') as csvfile3:
        csvwriter = csv.writer(csvfile3)
        csvwriter.writerows(forward)
    with open("midf"+csv_file, 'w') as csvfile4:
        csvwriter = csv.writer(csvfile4)
        csvwriter.writerows(midfielder)


./data1/Full Datasets - Opta/MLS/MLS 2018-2019.csv


In [6]:
print rows

[['Nikhil', 'COE', '2', '9.0'], ['Sanchit', 'COE', '2', '9.1']]
